In [1]:
#Loading dependencies
import requests
from pandas.io.json import json_normalize
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots


#Geo map of Cases per country globally
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

app =  dash.Dash(suppress_callback_exceptions = True)
server =  app.server

app.css.append_css({'external_url': 'https://codepen.io/amyoshino/pen/jzXypZ.css'})

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
    
#Reading in data
url = 'https://api.covid19api.com/summary'
r = requests.get(url)
dictr = r.json()
header = dictr['Countries']
df = json_normalize(header)
df = df.drop(df.index[[0,224]])
df = df.drop(columns = 'Slug')
df = df.sort_values(by=['TotalConfirmed'], ascending=False)
df.head()

/home/tshepo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:49: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



,Country,CountryCode,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
236,United States of America,US,0,1204351,0,71064,0,189791,2020-05-06T22:45:59Z
208,Spain,ES,0,219329,0,25613,0,123486,2020-05-06T22:45:59Z
108,Italy,IT,0,213013,0,29315,0,85231,2020-05-06T22:45:59Z
235,United Kingdom,GB,0,194990,0,29427,0,0,2020-05-06T22:45:59Z
74,France,FR,0,168935,0,25498,0,51803,2020-05-06T22:45:59Z


In [2]:
#Summing per column
tot_cases = df.TotalConfirmed.sum()
nw_cases = df.NewConfirmed.sum()
tot_deaths = df.TotalDeaths.sum()
new_deaths = df.NewDeaths.sum()
tot_recover = df.TotalRecovered.sum()
new_recover = df.NewRecovered.sum()
diff = tot_cases - nw_cases
diff2 = tot_recover - new_recover
diff3 = tot_deaths - new_deaths 

#Separating columns

#Corona  cases: Total and New
df_tot_cases = df.loc[:,['Country','TotalConfirmed']].sort_values(by='TotalConfirmed', ascending=False)
df_new_cases = df.loc[:,['Country','NewConfirmed']]

#Death Cases: Total and New 
df_tot_death = df.loc[:,['Country','TotalDeaths']]
df_new_death = df.loc[:,['Country','NewDeaths']]

#Recovery cases: Total and new
df_tot_recover = df.loc[:,['Country','TotalRecovered']]
df_new_recover = df.loc[:,['Country','NewRecovered']]

#Sorting cases by total
df_cases = df_tot_cases.merge(df_new_cases, on='Country').sort_values(by=['TotalConfirmed'],
                                                                      ascending=False)
df_death = df_tot_death.merge(df_new_death, on='Country').sort_values(by=['TotalDeaths'],
                                                                      ascending=False)
df_recover = df_tot_recover.merge(df_new_recover, on='Country').sort_values(by=['TotalRecovered'], 
                                                                            ascending=False)

# total cases indicator
fig1 = go.Figure(go.Indicator(
    
    value = tot_cases,
    delta = {'reference': diff},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 0, 'column': 0}))

fig1 = fig1.update_layout(
    template = {'data' : {'indicator': [{
        'title': {'text': "Total Cases"},
        'mode' : "number+delta+gauge",
        }]
                         }})

#Recoveries indicator
fig2 = go.Figure(go.Indicator(
    mode = "number+delta",
    value = tot_recover,
    delta = {'reference': diff2},
    domain = {'row': 0, 'column': 1}))
fig2 = fig2.update_layout(
    template = {'data' : {'indicator': [{
        'title': {'text': "Recoveries"}
        }]
                         }})

# total deaths indicator
fig3 = go.Figure(go.Indicator(
    
    value = tot_deaths,
    delta = {'reference': diff3},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 0, 'column': 0}))

fig3 = fig3.update_layout(
    template = {'data' : {'indicator': [{
        'title': {'text': "Total Deaths"},
        'mode' : "number+delta+gauge",
        }]
                         }})

#----- Feature Creation from existing data ----
df['Active Cases'] = df['TotalConfirmed'] - df['TotalRecovered'] - df['TotalDeaths']
df['Closed Cases'] = df['TotalRecovered'] + df['TotalDeaths']
df['Recovery Rate'] = (df['TotalRecovered'] / df['TotalConfirmed'])*100
df['Case Fatality Rate'] = (df['TotalDeaths'] / df['TotalConfirmed']) * 100

#---- Load in more datasets---->
url1 = 'https://api.covid19api.com/live/country/south-africa/status/confirmed/date/2020-03-21T13:13:30Z'
df1 = pd.read_json(url1, orient='columns')
url2 = 'https://api.covid19api.com/country/south-africa/status/confirmed'
df2 = pd.read_json(url2, orient='columns')
url3 = 'https://api.covid19api.com/country/south-africa/status/confirmed/live'
df3 = pd.read_json(url3, orient='columns')
url4 = 'https://covid19.soficoop.com/country/za'
url5 = 'https://api.covid19api.com/total/dayone/country/south-africa'
df5 = pd.read_json(url5, orient='columns')
df6 = pd.read_csv('datasets/covid.csv',delimiter=',')

#Daily Commulative dataframe 
df4 = requests.get(url4).json()
df4= pd.json_normalize(df4,record_path ='snapshots')


#SA commulative data
df_rsa = df6[df6.countriesAndTerritories == 'South_Africa'].reset_index()
fig_line = go.Figure()
fig_line.add_trace(go.Scatter(x = df4.timestamp,y= df4.active, name = 'Active Cases in SA'))
fig_line.update_layout(title = 'Commulative confirmed cases in SA as 17/03/2020')

df_rt = pd.read_csv('datasets/data-CiK32.csv')

#Effective reproduction
fig_rt = go.Figure()
fig_rt.add_trace(
    go.Scatter(
        y=df_rt['Possible low Rt'],
        x=df_rt.date,
        name= 'Possible low Rt'
    ))
fig_rt.add_trace(
    go.Scatter(
        y=df_rt['Most likely Rt'],
        x=df_rt.date,
       name= 'Most likely Rt'
    ))
fig_rt.add_trace(
    go.Scatter(
        y=df_rt['Possible High Rt'],
        x=df_rt.date,
        name= 'Possible High Rt'
    ))

fig_rt.update_layout(hovermode='x',
                     xaxis=dict(
                         rangeslider=dict(
                             visible = True),
        type='date'
    ))

# Provincial cases dataset
df_day = pd.read_csv('https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_confirmed.csv')
df_day = df_day.drop('source',axis = 1)
df_day = df_day.dropna()

#Commulative cases by province


fig_comm = go.Figure()
fig_comm.add_trace(
    go.Scatter(
        y=df_day['EC'],
        x=df_day.date,
        name= 'EC'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['WC'],
        x=df_day.date,
        name= 'WC'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['FS'],
        x=df_day.date,
        name= 'FS'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['GP'],
        x=df_day.date,
        name= 'GP'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['KZN'],
        x=df_day.date,
        name= "KZN"
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['NW'],
        x=df_day.date,
        name= 'NW'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['LP'],
        x=df_day.date,
        name= 'LP'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['MP'],
        x=df_day.date,
        name= 'MP'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['NC'],
        x=df_day.date,
        name= 'NC'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['UNKNOWN'],
        x=df_day.date,
        name='Unlocated'
    ))
# fig_comm.add_trace(
#     go.Scatter(
#         y=df_day['total'],
#         x=df_day.date,
#         name='Total Confirmed in SA'
#     ))

fig_comm.update_layout(hovermode='x',
                       title = 'Commulative confirmed cases by province',
                       xaxis=dict(
                         rangeslider=dict(
                             visible = True),
                           
       
                       
    ))





#Total confirmed world map
data = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z = df['TotalConfirmed'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(155,155,155)',
 width = 4,
 )  ),
 colorbar = dict(
 title = 'Number of cases'
 )
 ) ]
layout = dict(
 autosize=True,
margin={"r":0,"t":0,"l":0,"b":0}
    
)
fig = go.Figure(data = data, layout = layout)

# Active cases world map

data1 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z = df['Active Cases'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(255,255,255)',
 width = 10,
 )  ),
 colorbar = dict(
 title = 'Active cases'
 )
 ) ]
layout = dict(
 autosize=True,
 margin={"r":0,"t":0,"l":0,"b":0}   
)




fig1_ = go.Figure(data = data1, layout = layout)

#Closed cases world map
data2 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z =df['Closed Cases'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(245, 197, 66)',
 width = 10,
 )  ),
 colorbar = dict(
 title = 'Closed Cases'
 )
 ) ]
layout = dict(
 autosize=True,
 margin={"r":0,"t":0,"l":0,"b":0}   
)
fig2_ = go.Figure(data = data2, layout = layout)

#Recovery Rate World Map

data3 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z =df['Recovery Rate'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(455,455,455)',
 width = 10,
 )  ),
 colorbar = dict(
 title = 'Recovery Rate (%)'
 )
 ) ]
layout = dict(
 autosize=True,
 margin={"r":0,"t":0,"l":0,"b":0}   
)
fig3_ = go.Figure(data = data3, layout = layout)


#Case fatality rate world map

data4 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z =df['Case Fatality Rate'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(555,555,555)',
 width = 10,
 )  ),
 colorbar = dict(
 title = 'Case Fatality Rate (%)'
 )
 ) ]
layout = dict(
    autosize=True,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig4_ = go.Figure(data = data4, layout = layout)



In [3]:
# # Style sheet
# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# # App
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# # layout
# app.layout = html.Div([
    
#     #---------------------------header------------------------
    
#     html.Div([
#         html.Div([
#             html.H3('Header'),
            
#         ], className="four columns"),
        

#         html.Div([
#             html.H3('header'),
             
#         ], className="four columns"),
        
#         html.Div([
#             html.H3('header'),
            
#         ], className="four columns")
#     ], className="row"),
    
        
#     #----------------Indicators --------------
       
#      html.Div([
#         html.Div([
#             html.H3(''),
           
#         ], className="one columns"),
#         html.Div([
#             html.H3('Indicators'),
#             html.Div([
#                 html.H3(''),
                
            
#         ], className="four columns"),
        

#         html.Div([
#             html.H3(''),
             
#         ], className="four columns"),
        
#         html.Div([
#             html.H3(''),
            
#         ], className="four columns")
#     ], className="row"),
            
                      
#         ], className="ten columns"),

#         html.Div([
#             html.H3(''),
            
#         ], className="one columns"),
#     ], className="row"),
    
#     #---------------Main Visualisation-------------
    
#     html.Div([
#         html.Div([
#             html.H3('Active vs confirmed'),
            
#         ], className="four columns"),
        

#         html.Div([
#             html.H3('deaths'),
             
#         ], className="four columns"),
        
#         html.Div([
#             html.H3('Province death'),
            
#         ], className="four columns")
#     ], className="row"),
    
#     #------------ Provincial cases -------
    
#     html.Div([
#         html.Div([
#             html.H3('Cases by province'),
            
#         ], className="nine columns"),
        

#         html.Div([
#             html.H3('Provincial and pop'),
             
#         ], className="three columns"),
        
#     ], className="row"),
    
#     #---------- Tests and positives visualisation ----
    
#     html.Div([
#         html.Div([
#             html.H3('Average daily tests'),
            
#         ], className="four columns"),
        

#         html.Div([
#             html.H3('Average daily cases'),
             
#         ], className="four columns"),
        
#         html.Div([
#             html.H3('tests per positives'),
            
#         ], className="four columns")
#     ], className="row"),
    
#     #--------------- Infections, tests and positives visualisations ------
    
#     html.Div([
#         html.Div([
#             html.H3('Daily tests and infections'),
            
#         ], className="eight columns"),
        

#         html.Div([
#             html.H3('Positives'),
             
#         ], className="four columns"),
        
#     ], className="row"),
    
#     #----------More tests and infections visualisation ---------
    
#     html.Div([
#         html.Div([
#             html.H3('Infection rate vs tests'),
            
#         ], className="twelve columns")
#     ], className="row"),
    
#     #------------Provencial tests ---------
    
#     html.Div([
#         html.Div([
#             html.H3('Provincial tests'),
            
#         ], className="four columns"),
        

#         html.Div([
#             html.H3('tests per population'),
             
#         ], className="four columns"),
        
#         html.Div([
#             html.H3('pub vs private tests'),
            
#         ], className="four columns")
#     ], className="row"),
    
#     #-------------- Infection increase visualisation ---------
    
#     html.Div([
#         html.Div([
#             html.H3('test per population'),
            
#         ], className="four columns"),
        
#         html.Div([
#             html.H3('Infection increase'),
             
#         ], className="eight columns"),
       
#     ], className="row"),
    
#     #fOOTER
#      html.Div([
#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),
#     ], className="row"),
    
# ])

# if __name__ == '__main__':
#     app.run_server()


In [4]:


 dcc.Graph(

            id='figure1',
            figure=
              go.Figure(fig2)
            ),
dcc.Graph(

            id='figure2',
            figure=
              go.Figure(fig3)
            )
 

Graph(id='figure2', figure=Figure({
    'data': [{'delta': {'reference': 252359},
              'domain': {'column': 0, 'row': 0},
              'gauge': {'axis': {'visible': False}},
              'type': 'indicator',
              'value': 257093}],
    'layout': {'template': '...'}
}))

In [ ]:
# Style sheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# App
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# layout
app.layout = html.Div([
    
    #---------------------------header------------------------
    
    html.Div([
        html.Div([
            html.H3('Header'),
            
        ], className="four columns"),
        

        html.Div([
            html.H3('header'),
             
        ], className="four columns"),
        
        html.Div([
            html.H3('header'),
            
        ], className="four columns")
    ], className="row"),
    
        
    #----------------Indicators --------------
       

    html.Div([
        html.H3('Indicators'),
        html.Div([

            dcc.Graph(
                 id='figure',
                 figure=
                 go.Figure(fig1)
        ),



        ], className="four columns"),


        html.Div([
            dcc.Graph(
                id='figure1',
                figure=
                go.Figure(fig2)
        ),

        ], className="four columns"),

        html.Div([

            dcc.Graph(
                 id='figure2',
                 figure=
                 go.Figure(fig3)
        ),

        ], className="four columns")




    ], className="row"),

    #---------------Main Visualisation-------------
    
    html.Div([
        html.Div([
            html.H3('Active vs confirmed'),
            
        ], className="four columns"),
        

        html.Div([
            html.H3('deaths'),
             
        ], className="four columns"),
        
        html.Div([
            html.H3('Province death'),
            
        ], className="four columns")
    ], className="row"),
    
    #------------ Provincial cases -------
    
    html.Div([
        html.Div([
            html.H3('Cases by province'),
            
        ], className="nine columns"),
        

        html.Div([
            html.H3('Provincial and pop'),
             
        ], className="three columns"),
        
    ], className="row"),
    
    #---------- Tests and positives visualisation ----
    
    html.Div([
        html.Div([
            html.H3('Average daily tests'),
            
        ], className="four columns"),
        

        html.Div([
            html.H3('Average daily cases'),
             
        ], className="four columns"),
        
        html.Div([
            html.H3('tests per positives'),
            
        ], className="four columns")
    ], className="row"),
    
    #--------------- Infections, tests and positives visualisations ------
    
    html.Div([
        html.Div([
            html.H3('Daily tests and infections'),
            
        ], className="eight columns"),
        

        html.Div([
            html.H3('Positives'),
             
        ], className="four columns"),
        
    ], className="row"),
    
    #----------More tests and infections visualisation ---------
    
    html.Div([
        html.Div([
            html.H3('Infection rate vs tests'),
            
        ], className="twelve columns")
    ], className="row"),
    
    #------------Provencial tests ---------
    
    html.Div([
        html.Div([
            html.H3('Provincial tests'),
            
        ], className="four columns"),
        

        html.Div([
            html.H3('tests per population'),
             
        ], className="four columns"),
        
        html.Div([
            html.H3('pub vs private tests'),
            
        ], className="four columns")
    ], className="row"),
    
    #-------------- Infection increase visualisation ---------
    
    html.Div([
        html.Div([
            html.H3('test per population'),
            
        ], className="four columns"),
        
        html.Div([
            html.H3('Infection increase'),
             
        ], className="eight columns"),
       
    ], className="row"),
    
    #fOOTER
     html.Div([
        html.Div([
            html.H3('Footer'),
            
        ], className="six columns"),

        html.Div([
            html.H3('Footer'),
            
        ], className="six columns"),
    ], className="row"),
    
])

if __name__ == '__main__':
    app.run_server()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/May/2020 00:55:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2020 00:55:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2020 00:55:18] "GET /_dash-layout HTTP/1.1" 200 -
